<a href="https://colab.research.google.com/github/marcoweb/colab-indexacao/blob/main/ColetaDeArtigos_Embrapa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importação das bibiotecas

In [ ]:
import os
import urllib
import pandas
import requests
from bs4 import BeautifulSoup

Busca os dados das revistas na base de dados da embrapa e baixa os arquivos xls com as informações dos artigos

In [ ]:
embrapaFilesPath = 'embrapaSource/'
if not os.path.exists(embrapaFilesPath):
  os.mkdir(embrapaFilesPath)
dfRevistas = pandas.read_excel('revistas.xlsx')
for item in dfRevistas.to_dict('records'):
  embrapaUrl = 'https://www.bdpa.cnptia.embrapa.br/consulta/busca?b=ad&busca=((fonte-imprenta:%22'+urllib.parse.quote(item['revista'])+'%22))&qFacets=(((fonte-imprenta:%22'+urllib.parse.quote(item['revista'])+'%22)))%20%20AND%20((idioma:%22Ingl%C3%AAs%22))&biblioteca=vazio&sort=&paginacao=t&paginaAtual=1&ig=tn'
  fileData = requests.get(embrapaUrl).content
  with open(embrapaFilesPath + str(item['id']) + '.xls', 'wb') as handler:
    handler.write(fileData)


Lê os dados dos arquivos xls contendo as informações dos artigos da Embrapa criando um dataframe com as informações relevantes (Título, Ano de Publicação, Thesaurus Nal)

In [ ]:
dfValidPapers = pandas.DataFrame();
for item in dfRevistas.to_dict('records'):
  fileName = embrapaFilesPath + str(item['id']) + '.xls'
  dfTemp = pandas.read_excel(fileName, skiprows=4)
  dfDadosEmbrapa = dfTemp[(dfTemp['Thesaurus Nal'].notna()) & (dfTemp['Ano de publicação'] >= 2017)][['Título', 'Thesaurus Nal', 'Ano de publicação']].sort_values(by=['Ano de publicação'])
  dfValidPapers = pandas.concat([dfValidPapers, dfDadosEmbrapa], ignore_index=True)
dfValidPapers['Título'] = dfValidPapers['Título'].apply(lambda x: " ".join(x.split()).removesuffix('.'))

Acessa a página no Redalyc das revistas coletando as urls para acesso às coleções de artigos por ano de pulicação

In [ ]:
listUrls = {}
for revista in dfRevistas.to_dict('records'):
  redalycUrl = 'https://www.redalyc.org/revista.oa?id=' + str(revista['id'])
  req = requests.get(redalycUrl)
  html = req.text
  soup = BeautifulSoup(html, 'html.parser')
  listUrls[str(revista['id'])] = {}
  for item in soup.find_all('label', attrs={'class': 'anio-numeros'}):
    year = item.find('span').text
    if year not in listUrls[str(revista['id'])].keys():
      if int(year) >= 2017:
        listUrls[str(revista['id'])][year] = item.find('a')['href']

Verifica se os artigos existentes no redalyc contem os metadados selecionados na Embrapa fazendo o download do XML quando disponível.

In [ ]:
basePath = 'files/'
if not os.path.exists(basePath):
  os.mkdir(basePath)

goldIndexing = []
for id, data in listUrls.items():
  for year, url in data.items():
    dfValidPapersByYear = dfValidPapers[dfValidPapers['Ano de publicação'] == int(year)]
    if dfValidPapersByYear.size > 0:
      redalycContentHtml = BeautifulSoup(requests.get(url).text, 'html.parser')
      for article in redalycContentHtml.find_all('div', attrs={'class': 'articuloN'}):
        articleName = " ".join(article.find('b', attrs={'class': 'tituloArt'}).text.split())
        if dfValidPapersByYear.isin([articleName]).any().any():
          for link in article.find('div', attrs={'class': 'links'}).find_all('a'):
            if link['href'].endswith('.xml'):
              fileId = 'D' + str((len(goldIndexing) + 1))
              fileData = requests.get('https://www.redalyc.org/' + link['href']).content
              xml = BeautifulSoup(fileData, 'xml')
              if xml.find('article')['xml:lang'] == 'en':
                fileName = basePath + fileId + '-' + articleName.replace('/', '_') + '.xml'
                with open(fileName, 'wb') as handler:
                  handler.write(fileData)
                goldIndexing.append({'id': fileId, 'title': articleName, 'indexing': dfValidPapersByYear.loc[dfValidPapersByYear['Título'] == articleName]['Thesaurus Nal'].item().removesuffix('.').replace(';', ',')})

Cria dos arquivos com a lista de artigos baixados e gold indexing.

In [ ]:
with open(basePath + "gold_indexing.txt", "w") as gi:
  with open(basePath + "_articles.txt", "w") as articles:
    for item in goldIndexing:
      gi.write(item['id'] + ': ' + item['indexing'] + "\n")
      articles.write(item['id'] + ': ' + item['title'] + "\n")

Compacta os arquivos para download

In [ ]:
import zipfile
import glob

with zipfile.ZipFile('corpus.zip', 'w') as zip:
   for file in glob.glob(basePath + '*'):
        zip.write(file)